## Goal
Transform master dataset (created in Notebook 1) into BI-ready, normalized tables:
1. dim_patients
2. dim_admissions
3. fact_labs
4. fact_diagnoses
5. fact_medications
6. fact_outcomes (mortality, LOS)

#### Importing + Loading Master File

In [1]:
import pandas as pd
import numpy as np

# Load the master dataset created in Notebook 1
df = pd.read_csv("Data/Generated/CORE_MERGED.csv")

df.head()

,ROW_ID_x,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,ICU_LOS_hours
0,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,243653.0,carevue,NICU,NICU,56.0,56.0,2138-07-17 21:20:07,2138-07-17 23:32:21,0.0918,2.203889
1,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,211552.0,carevue,MICU,MICU,12.0,12.0,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646,145.549444
2,3,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,...,294638.0,carevue,MICU,MICU,52.0,52.0,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785,40.283333
3,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,214757.0,carevue,NICU,NICU,56.0,56.0,2103-02-02 06:04:24,2103-02-02 08:06:00,0.0844,2.026667
4,5,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,...,228232.0,carevue,SICU,SICU,33.0,33.0,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729,88.150000


In [2]:
# Load the master dataset created in Notebook 2
df_features = pd.read_csv("Data/Generated/clean_patient_level_features.csv")

In [3]:
df_features.head()

,AGE,LOS_hours,ICU_LOS_hours,charlson,lactate,platelets,creatinine,bun,sodium,potassium,bilirubin,hemoglobin,SUBJECT_ID,HOSPITAL_EXPIRE_FLAG
0,0,92.733333,2.203889,1,2.000000,153.5,1.00,20.00,139.0,4.166667,0.70,10.883333,2,0
1,76,258.833333,145.549444,0,4.871429,271.0,2.65,44.75,139.2,4.360000,0.80,9.850000,3,0
2,48,186.216667,40.283333,1,2.100000,204.0,0.50,9.50,138.0,3.200000,2.05,10.500000,4,0
3,0,55.733333,2.026667,1,2.000000,309.0,1.00,20.00,139.0,4.166667,0.70,10.883333,5,0
4,66,392.750000,88.150000,0,2.640000,322.5,10.85,63.50,136.0,5.100000,0.20,9.080000,6,0


In [4]:
# Load the diagnosis dataframe created in Notebook 2
df_diagnoses = pd.read_csv("Data/All_Diagnoses.csv")

In [5]:
df_diagnoses.head()

,SUBJECT_ID,ICD9_CODE
0,2,"['V3001', 'V053', 'V290']"
1,3,"['2639', '6826', '4280', '41071', '4254', '584..."
2,4,"['E9317', '042', '1363', '7994', '2763', '7907..."
3,5,"['V053', 'V290', 'V3000']"
4,6,"['40391', '4440', '9972', '2766', '2767', '285..."


#### Combined DataFrame

In [6]:
combined_df = pd.merge(
    pd.merge(df, df_features, on='SUBJECT_ID'), 
    df_diagnoses, 
    on='SUBJECT_ID', 
    how='left'
)

In [7]:
combined_df.head()

,ROW_ID_x,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,lactate,platelets,creatinine,bun,sodium,potassium,bilirubin,hemoglobin,HOSPITAL_EXPIRE_FLAG_y,ICD9_CODE
0,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,2.000000,153.5,1.00,20.00,139.0,4.166667,0.70,10.883333,0,"['V3001', 'V053', 'V290']"
1,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,4.871429,271.0,2.65,44.75,139.2,4.360000,0.80,9.850000,0,"['2639', '6826', '4280', '41071', '4254', '584..."
2,3,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,...,2.100000,204.0,0.50,9.50,138.0,3.200000,2.05,10.500000,0,"['E9317', '042', '1363', '7994', '2763', '7907..."
3,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,2.000000,309.0,1.00,20.00,139.0,4.166667,0.70,10.883333,0,"['V053', 'V290', 'V3000']"
4,5,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,...,2.640000,322.5,10.85,63.50,136.0,5.100000,0.20,9.080000,0,"['40391', '4440', '9972', '2766', '2767', '285..."


#### Create dim_patients

In [8]:
# Compute Age
df['DOB'] = pd.to_datetime(df['DOB'], errors='coerce')
df['ADMITTIME'] = pd.to_datetime(df['ADMITTIME'], errors='coerce')

df["AGE"] = (df["ADMITTIME"].dt.year - df["DOB"].dt.year).clip(0,110)
df["AGE"].describe()

count    13700.000000
mean        51.297883
std         31.100375
min          0.000000
25%         32.000000
50%         58.000000
75%         75.000000
max        110.000000
Name: AGE, dtype: float64

In [9]:
df.columns

Index(['ROW_ID_x', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA', 'LOS_hours', 'ROW_ID_y',
       'GENDER', 'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN', 'EXPIRE_FLAG', 'ROW_ID',
       'ICUSTAY_ID', 'DBSOURCE', 'FIRST_CAREUNIT', 'LAST_CAREUNIT',
       'FIRST_WARDID', 'LAST_WARDID', 'INTIME', 'OUTTIME', 'LOS',
       'ICU_LOS_hours', 'AGE'],
      dtype='object')

In [10]:
dim_patients = df[[
    "SUBJECT_ID", "GENDER", "DOB", "AGE", 
    "ETHNICITY", "INSURANCE"
]].drop_duplicates()

dim_patients.to_csv("Visualization/bi_dim_patients.csv", index=False)

#### Create dim_admissions

In [11]:
dim_admissions = df[[
    "SUBJECT_ID", "HADM_ID", "ADMITTIME", "DISCHTIME", 
    "ADMISSION_TYPE", "ADMISSION_LOCATION", 
    "DISCHARGE_LOCATION", "LOS"
]].drop_duplicates()

dim_admissions.to_csv("Visualization/bi_dim_admissions.csv", index=False)

#### Create fact_labs

In [12]:
fact_labs = combined_df[[
    "SUBJECT_ID", "creatinine", "lactate",
    "hemoglobin", "bun", "platelets"
]]

fact_labs.to_csv("Visualization/bi_fact_labs.csv", index=False)

#### Create fact_diagnoses

In [13]:
fact_diagnoses = combined_df[[
    "SUBJECT_ID", "HADM_ID", 
    "DIAGNOSIS", "ICD9_CODE"
]]

fact_diagnoses.to_csv("Visualization/bi_fact_diagnoses.csv", index=False)

#### Create fact_medications

In [14]:
prescriptions_df = pd.read_csv("Data/PRESCRIPTIONS_sorted.csv", low_memory=False)

In [15]:
prescriptions_df.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTDATE', 'ENDDATE',
       'DRUG_TYPE', 'DRUG', 'DRUG_NAME_POE', 'DRUG_NAME_GENERIC',
       'FORMULARY_DRUG_CD', 'GSN', 'NDC', 'PROD_STRENGTH', 'DOSE_VAL_RX',
       'DOSE_UNIT_RX', 'FORM_VAL_DISP', 'FORM_UNIT_DISP', 'ROUTE'],
      dtype='object')

In [16]:
prescriptions_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2968761,2,163353,243653.0,2138-07-18 00:00:00,2138-07-20 00:00:00,MAIN,NEO*IV*Gentamicin,NaN,NaN,GENT10I,009298,6.332302e+10,10mg/mL-2mL,15.5,mg,0.775,VIAL,IV
1,2968759,2,163353,243653.0,2138-07-18 00:00:00,2138-07-20 00:00:00,BASE,Syringe (Neonatal) *D5W*,NaN,NaN,NEOSYRD5W,NaN,0.000000e+00,1 Syringe,2.4,ml,2.4,ml,IV
2,2968762,2,163353,243653.0,2138-07-18 00:00:00,2138-07-21 00:00:00,MAIN,Ampicillin Sodium,NaN,NaN,AMP500I,008937,6.332304e+10,500mg Vial,500,mg,1,VIAL,IV
3,2968760,2,163353,243653.0,2138-07-18 00:00:00,2138-07-21 00:00:00,BASE,Send 500mg Vial,NaN,NaN,AMPVL,NaN,0.000000e+00,Send 500mg Vial,1,VIAL,1,VIAL,IV
4,1213835,4,185777,294638.0,2191-03-16 00:00:00,2191-03-23 00:00:00,MAIN,Guaifenesin-Codeine Phosphate,Guaifenesin-Codeine Phosphate,Guaifenesin-Codeine Phosphate,GGAC5L,045667,3.186741e+07,5ML UDCUP,5-10,ml,1-2,UDCUP,PO


In [17]:
fact_medications = prescriptions_df[[
    "SUBJECT_ID", "HADM_ID",
    "DRUG", "FORMULARY_DRUG_CD"
]]

fact_medications.to_csv("Visualization/bi_fact_medications.csv", index=False)

#### Create fact_outcomes (mortality, LOS, readmission)

In [18]:
fact_outcomes = df[[
    "SUBJECT_ID", "HADM_ID",
    "EXPIRE_FLAG", "LOS"
]]

fact_outcomes.to_csv("Visualization/bi_fact_outcomes.csv", index=False)

#### Summary Ready for Power BI

In [19]:
print("Exported BI tables:")
print("bi_dim_patients.csv")
print("bi_dim_admissions.csv")
print("bi_fact_labs.csv")
print("bi_fact_diagnoses.csv")
print("bi_fact_medications.csv")
print("bi_fact_outcomes.csv")

Exported BI tables:
bi_dim_patients.csv
bi_dim_admissions.csv
bi_fact_labs.csv
bi_fact_diagnoses.csv
bi_fact_medications.csv
bi_fact_outcomes.csv
